In [1]:
# import dependencies
import datetime as dt
import pandas as pd
import requests, io, zipfile, os

In [8]:
# format of url = "https://s3.amazonaws.com/tripdata/201306-citibike-tripdata.zip"
url_base = 'https://s3.amazonaws.com/tripdata/'

root = 'Resources'

result_files = []

In [9]:
# files set up by month, get by 4 digit year and 2 digit month
d = dt.date.today()
# url_year = d.year
# url_month = d.month

url_year = 2013
url_month = 6
url_file_prefix = f'{url_year:04d}{url_month:02d}'
# url_file_prefix

# directory files same through 2016, changed for 2017
url_file_2016 = '-citibike-tripdata.zip'
url_file_2017 = '-citibike-tripdata.csv.zip'

# substring = 'citibike-tripdata.csv'

# build the proper url to get data from files in the directory
if url_year >= 2017:
    url = url_base + url_file_prefix + url_file_2017
else:
    url = url_base + url_file_prefix + url_file_2016
print(url) 

r = requests.get(url)
unzipfolder = zipfile.ZipFile(io.BytesIO(r.content))
unzipfolder.extractall(root)

for file in os.listdir(root):
    if os.path.isfile(os.path.join(root, file)):
        result_file = os.path.join(root, file)
        print(result_file)
        result_files.append(result_file)

https://s3.amazonaws.com/tripdata/201306-citibike-tripdata.zip
Resources\201306-citibike-tripdata.csv
Resources\201703-citibike-tripdata.csv


In [10]:
result_files



['Resources\\201306-citibike-tripdata.csv',
 'Resources\\201703-citibike-tripdata.csv']

In [73]:
result_files

['2013-07 - Citi Bike trip data.csv', '201306-citibike-tripdata.csv']

In [24]:
df2 = pd.read_csv(result_files[0])
df2

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,695,2013-06-01 00:00:01,2013-06-01 00:11:36,444,Broadway & W 24 St,40.742354,-73.989151,434.0,9 Ave & W 18 St,40.743174,-74.003664,19678,Subscriber,1983.0,1
1,693,2013-06-01 00:00:08,2013-06-01 00:11:41,444,Broadway & W 24 St,40.742354,-73.989151,434.0,9 Ave & W 18 St,40.743174,-74.003664,16649,Subscriber,1984.0,1
2,2059,2013-06-01 00:00:44,2013-06-01 00:35:03,406,Hicks St & Montague St,40.695128,-73.995951,406.0,Hicks St & Montague St,40.695128,-73.995951,19599,Customer,NaN,0
3,123,2013-06-01 00:01:04,2013-06-01 00:03:07,475,E 15 St & Irving Pl,40.735243,-73.987586,262.0,Washington Park,40.691782,-73.973730,16352,Subscriber,1960.0,1
4,1521,2013-06-01 00:01:22,2013-06-01 00:26:43,2008,Little West St & 1 Pl,40.705693,-74.016777,310.0,State St & Smith St,40.689269,-73.989129,15567,Subscriber,1983.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
577698,925,2013-06-30 23:59:27,2013-07-01 00:14:52,509,9 Ave & W 22 St,40.745497,-74.001971,462.0,W 22 St & 10 Ave,40.746920,-74.004519,15250,Subscriber,1964.0,1
577699,279,2013-06-30 23:59:36,2013-07-01 00:04:15,116,W 17 St & 8 Ave,40.741776,-74.001497,494.0,W 26 St & 8 Ave,40.747348,-73.997236,19411,Customer,NaN,0
577700,161,2013-06-30 23:59:33,2013-07-01 00:02:14,443,Bedford Ave & S 9 St,40.708531,-73.964090,282.0,Kent Ave & S 11 St,40.707645,-73.968415,18910,Subscriber,1972.0,2
577701,909,2013-06-30 23:59:47,2013-07-01 00:14:56,509,9 Ave & W 22 St,40.745497,-74.001971,462.0,W 22 St & 10 Ave,40.746920,-74.004519,16108,Customer,NaN,0


In [ ]:
# for result in result_files:
#     df2 = pd.read_csv(url2, compression = 'zip', engine ='python')

In [26]:
df3 = pd.read_csv(result_files[1])
df3

,Trip Duration,Start Time,Stop Time,Start Station ID,Start Station Name,Start Station Latitude,Start Station Longitude,End Station ID,End Station Name,End Station Latitude,End Station Longitude,Bike ID,User Type,Birth Year,Gender
0,1893,2017-03-01 00:00:32,2017-03-01 00:32:06,2009,Catherine St & Monroe St,40.711174,-73.996826,527,E 33 St & 2 Ave,40.744023,-73.976056,27291,Subscriber,1973.0,2
1,223,2017-03-01 00:01:09,2017-03-01 00:04:53,127,Barrow St & Hudson St,40.731724,-74.006744,284,Greenwich Ave & 8 Ave,40.739017,-74.002638,19387,Subscriber,1985.0,2
2,1665,2017-03-01 00:01:27,2017-03-01 00:29:12,174,E 25 St & 1 Ave,40.738177,-73.977387,307,Canal St & Rutgers St,40.714275,-73.989900,15809,Subscriber,1988.0,1
3,100,2017-03-01 00:01:29,2017-03-01 00:03:10,316,Fulton St & William St,40.709560,-74.006536,306,Cliff St & Fulton St,40.708235,-74.005301,18956,Subscriber,1991.0,1
4,1229,2017-03-01 00:01:33,2017-03-01 00:22:02,536,1 Ave & E 30 St,40.741444,-73.975361,259,South St & Whitehall St,40.701221,-74.012342,25728,Subscriber,1963.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
727660,261,2017-03-31 23:57:12,2017-04-01 00:01:34,482,W 15 St & 7 Ave,40.739355,-73.999318,509,9 Ave & W 22 St,40.745497,-74.001971,28366,Subscriber,1983.0,1
727661,119,2017-03-31 23:59:29,2017-04-01 00:01:28,3379,E 103 St & Lexington Ave,40.790305,-73.947558,3425,2 Ave & E 104 St,40.789211,-73.943708,16315,Subscriber,1987.0,1
727662,451,2017-03-31 23:59:42,2017-04-01 00:07:13,3224,W 13 St & Hudson St,40.739974,-74.005139,3259,9 Ave & W 28 St,40.749370,-73.999234,27996,Subscriber,1974.0,1
727663,1032,2017-03-31 23:59:49,2017-04-01 00:17:01,351,Front St & Maiden Ln,40.705310,-74.006126,302,Avenue D & E 3 St,40.720828,-73.977932,21097,Subscriber,1978.0,1
